# PyTorch & Plant Seedling Classification Tutorial

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Data Extraction & Preprocessing

In [2]:
df = pd.read_csv('csv/data.csv', sep=';', index_col=0)

In [3]:
df.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p3063,p3064,p3065,p3066,p3067,p3068,p3069,p3070,p3071,label
0,65,56,60,68,58,61,64,53,59,68,...,96,77,62,95,76,61,103,83,65,Black-grass
1,53,40,36,50,37,34,50,40,39,54,...,47,40,41,50,41,42,49,42,44,Black-grass
2,148,152,154,165,176,183,172,178,186,172,...,59,34,24,71,54,43,78,60,48,Black-grass
3,100,86,66,85,65,46,87,73,51,84,...,91,70,42,101,70,41,99,70,43,Black-grass
4,66,49,35,98,82,61,129,114,88,133,...,105,91,70,84,70,54,79,63,50,Black-grass


---

In [4]:
x = df.drop('label', axis=1)
y = df.label

In [5]:
x.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p3062,p3063,p3064,p3065,p3066,p3067,p3068,p3069,p3070,p3071
0,65,56,60,68,58,61,64,53,59,68,...,66,96,77,62,95,76,61,103,83,65
1,53,40,36,50,37,34,50,40,39,54,...,41,47,40,41,50,41,42,49,42,44
2,148,152,154,165,176,183,172,178,186,172,...,20,59,34,24,71,54,43,78,60,48
3,100,86,66,85,65,46,87,73,51,84,...,48,91,70,42,101,70,41,99,70,43
4,66,49,35,98,82,61,129,114,88,133,...,73,105,91,70,84,70,54,79,63,50


In [6]:
y.head()

0    Black-grass
1    Black-grass
2    Black-grass
3    Black-grass
4    Black-grass
Name: label, dtype: object

---

In [7]:
l_encoder = LabelEncoder()
y = l_encoder.fit_transform(y)

In [8]:
x = np.array(x).reshape(-1, 32, 32, 3)
x = np.transpose(x, (0, 3, 1, 2))

---

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    random_state=42,
                                                    test_size=0.2)

---

In [10]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Net parameters

In [11]:
EPOCHES = 50
BATCH_SIZE = 32
NUM_CLASSES = 12

## Batch Retrieval

In [12]:
def get_batch(x, y=None, batch_size=None):
    if batch_size is None:
        batch_size = BATCH_SIZE
    
    selected = np.random.randint(0, x.shape[0], batch_size)
    
    if y is None:
        return x[selected]
    else:
        return x[selected], y[selected]

## Net Architecture

In [13]:
class MainNet(nn.Module):
    def __init__(self):
        super(MainNet, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout_1 = nn.Dropout2d(p=0.3)
        self.conv_5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.dense_1 = nn.Linear(3 * 3 * 512, 1024)
        self.dense_2 = nn.Linear(1024, 256)
        self.dense_3 = nn.Linear(256, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool_1(F.relu(self.conv_3(x)))
        x = self.pool_2(F.relu(self.conv_4(x)))
        x = F.relu(self.dropout_1(x))
        x = F.relu(self.conv_5(x))
        x = x.view(-1, 3 * 3 * 512)
        x = F.relu(self.dense_1(x))
        x = F.relu(self.dense_2(x))
        x = F.dropout(x, training=self.training)
        x = self.dense_3(x)
        return x

---

In [14]:
net = MainNet()

loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Net Training

In [15]:
for epoch in range(EPOCHES):
    total_loss = .0
    for i in range(x_train.shape[0] // BATCH_SIZE):
        x_batch, y_batch = get_batch(x_train, y_train, BATCH_SIZE)
        optimizer.zero_grad()
        y_pred = net(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 5 == 4 or epoch == 0:
        print('EPOCH\t {0:3d}\t LOSS: {1:.3f}'.format(epoch + 1, total_loss / 100))

EPOCH	   1	 LOSS: 3.062
EPOCH	   5	 LOSS: 1.353
EPOCH	  10	 LOSS: 0.714
EPOCH	  15	 LOSS: 0.443
EPOCH	  20	 LOSS: 0.273
EPOCH	  25	 LOSS: 0.196
EPOCH	  30	 LOSS: 0.116
EPOCH	  35	 LOSS: 0.098
EPOCH	  40	 LOSS: 0.066
EPOCH	  45	 LOSS: 0.052
EPOCH	  50	 LOSS: 0.034


## Net Accuracy

In [16]:
def total_accuracy(net, x_test, y_test):
    correct = 0
    
    with torch.no_grad():
        for i in range(len(x_test)):
            logits = net(x_test[i: i + 1])
            _, pred = torch.max(logits, 1)
            if pred == y_test[i]:
                correct += 1

    print('TOTAL ACCURACY OF THE NET\n\t{0:.3f}'.format(correct / x_test.shape[0]))

In [22]:
def class_accuracy(net, x_test, y_test):
    correct = list(0 for i in range(NUM_CLASSES))
    total = list(0 for i in range(NUM_CLASSES))

    with torch.no_grad():
        for i in range(len(x_test)):
            logits = net(x_test[i: i + 1])
            _, pred = torch.max(logits, 1)
            mask = (pred == y_test[i])
            correct[y_test[i]] += mask.item()
            total[y_test[i]] += 1

    print('CLASSWISE ACCURACY\n')
    for i in range(NUM_CLASSES):
        print('\tACCURACY OF CLASS {0:2d}: {1:.3f}%'.format(i, 100 * correct[i] / total[i]))

---

In [27]:
total_accuracy(net, x_test, y_test)

TOTAL ACCURACY OF THE NET
	0.805


In [24]:
class_accuracy(net, x_test, y_test)

CLASSWISE ACCURACY

	ACCURACY OF CLASS  0: 26.027%
	ACCURACY OF CLASS  1: 91.000%
	ACCURACY OF CLASS  2: 78.462%
	ACCURACY OF CLASS  3: 91.391%
	ACCURACY OF CLASS  4: 56.410%
	ACCURACY OF CLASS  5: 82.524%
	ACCURACY OF CLASS  6: 89.437%
	ACCURACY OF CLASS  7: 67.442%
	ACCURACY OF CLASS  8: 85.586%
	ACCURACY OF CLASS  9: 74.242%
	ACCURACY OF CLASS 10: 90.517%
	ACCURACY OF CLASS 11: 74.737%


## Saving the Net

In [20]:
torch.save(net.state_dict(), 'MainNet')